In [ ]:
#removing missing data
df.drop([1,2,3]) #drops rows by default
df.drop('A', axis=1)
df.dropna(subset=["B"])

In [ ]:
# Drop the Latitude and Longitude columns from volunteer
volunteer_cols = volunteer.drop(['Latitude','Longitude'], axis=1)

# Drop rows with missing category_desc values from volunteer_cols
volunteer_subset = volunteer_cols.dropna(subset=['category_desc'])

In [ ]:
#working with data types
# Convert the hits column to type int
volunteer["hits"] = volunteer["hits"].astype('int')

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

knn = KNeighborsClassifier()

# Fit the knn model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test,y_test))

In [ ]:
#scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df),
                        columns=df.columns)

In [ ]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Create the scaler
scaler = StandardScaler()

# Subset the DataFrame you want to scale 
wine_subset = wine[['Ash', 'Alcalinity of ash', 'Magnesium']]

# Apply the scaler to wine_subset
wine_subset_scaled = scaler.fit_transform(wine_subset)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Instantiate a StandardScaler
scaler = StandardScaler()

# Scale the training and test features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train_scaled, y_train)

# Score the model on the test data
print(knn.score(X_test_scaled, y_test))

In [4]:
#encoding categorical variables
user=[1,2,3,4]
subscribed = ['y','n','n','y']
fav_color = ['blue','green','orange','green']
df = pd.DataFrame({'user': user,"subscribed":subscribed,'fav_color':fav_color})
df.head()

,user,subscribed,fav_color
0,1,y,blue
1,2,n,green
2,3,n,orange
3,4,y,green


In [5]:
#using pandas
df['sub_enc'] = df['subscribed'].apply(lambda val:1 if val=="y" else 0)

In [8]:
#encoding via sklearn
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['sub_enc_le'] = le.fit_transform(df['subscribed'])
df.head()

,user,subscribed,fav_color,sub_enc,sub_enc_le
0,1,y,blue,1,1
1,2,n,green,0,0
2,3,n,orange,0,0
3,4,y,green,1,1


In [9]:
#one hot encoding
pd.get_dummies(df['fav_color'])

,blue,green,orange
0,1,0,0
1,0,1,0
2,0,0,1
3,0,1,0


In [ ]:
# Set up the LabelEncoder object
enc = LabelEncoder()

# Apply the encoding to the "Accessible" column
hiking['Accessible_enc'] = enc.fit_transform(hiking['Accessible'])

# Compare the two columns
print(hiking[['Accessible', 'Accessible_enc']].head())

In [ ]:
#engineering numerical features
temps['mean'] = temps.loc[:,"day1":'day3'].mean(axis=1)
purchases['date_converted'] = pd.to_datetime(purchases['date'])
purchases['month'] = purchases.date_converted.dt.month


In [10]:
#engineering text features
import re
my_string = 'temperature:75.6 F'
temp = re.search('\d+\.\d+', my_string)
print(float(temp.group(0)))

75.6


In [ ]:
#vectorizing text
#TF/IDF:Vectorizes words based upon importance(Term Frequency, Inverse Document Frequency)
from sklearn.feature_extraction.text import TfidVectorizer
tfidf_vec = TfidVectorizer()
text_tfidf = tfidf_vec.fit_transform(documents)

In [ ]:
# Write a pattern to extract numbers and decimals
def return_mileage(length):
    
    # Search the text for matches
    mile = re.search('\d+\.\d+', length)
    
    # If a value is returned, use group(0) to return the found value
    if mile is not None:
        return float(mile.group(0))
        
# Apply the function to the Length column and take a look at both columns
hiking["Length_num"] = hiking['Length'].apply(return_mileage)
print(hiking[["Length", "Length_num"]].head())

In [ ]:
# Split the dataset according to the class distribution of category_desc
y = volunteer["category_desc"]
X_train, X_test, y_train, y_test = train_test_split(text_tfidf.toarray(),y , stratify=y, random_state=42)

# Fit the model to the training data
nb.fit(X_train, y_train)

# Print out the model's accuracy
print(nb.score(X_test, y_test))